## Import Necessary Libraries

In [11]:
import pandas as pd

## Data Loading & Basic Stats

In [12]:
df = pd.read_csv('dataset/Facial Skin Condition Dataset.csv') #loading the dataset
df.head() #displaying first 5 rows

,id,gender,image front,image right side,image left side
0,1,male,1/front.png,1/right-side.png,1/left-side.png
1,2,female,2/front.png,2/right-side.png,2/left-side.png
2,3,female,3/front.png,3/right-side.png,3/left-side.png
3,4,female,4/front.png,4/right-side.png,4/left-side.png
4,5,male,5/front.png,5/right-side.png,5/left-side.png


In [13]:
df.shape #How many rows and colums

(15, 5)

In [14]:
df.info() #obtaining summary information about the dataset

<class 'pandas.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   id                15 non-null     int64
 1   gender            15 non-null     str  
 2   image front       15 non-null     str  
 3   image right side  15 non-null     str  
 4   image left side   15 non-null     str  
dtypes: int64(1), str(4)
memory usage: 732.0 bytes


In [15]:
df.dtypes #checking the datatypes of each column

id                  int64
gender                str
image front           str
image right side      str
image left side       str
dtype: object